In [2]:
import gym
from gym import spaces
import numpy as np

class WarehouseOrientationEnv(gym.Env):
    """
    Magazyn NxN z orientacją robota:
      - Stan: [robot_x, robot_y, target_x, target_y, orientation_index]
      - Akcje: 0=obrót w lewo o alpha, 1=obrót w prawo o alpha, 2=ruch do przodu
    """
    def __init__(self, n=5, alpha=90, max_steps=100):
        super().__init__()  # Wywołanie init z gym.Env

        self.n = n
        self.alpha = alpha
        self.max_steps = max_steps
        
        # Ilu "kroków" orientacji (np. 360 / 90 = 4)
        assert 360 % alpha == 0, "Kąt alpha musi dzielić 360 (np. 90, 45, 60)."
        self.num_orientations = 360 // alpha

        # Przestrzeń akcji: 3 akcje
        self.action_space = spaces.Discrete(3)

        # Przestrzeń stanów (Box z 5 wymiarami):
        #   robot_x, robot_y, target_x, target_y, orientation_index
        low = np.array([0, 0, 0, 0, 0], dtype=np.float32)
        high = np.array([n-1, n-1, n-1, n-1, self.num_orientations - 1], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        # Pola do przechowywania stanu
        self.robot_x = 0
        self.robot_y = 0
        self.target_x = 0
        self.target_y = 0
        self.orientation = 0  # indeks 0..(num_orientations-1)
        self.current_step = 0

        # Dodatkowe flagi
        self.terminated = False
        self.reward = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)  # możliwe wywołanie, jeśli Gym >=0.26
        self.current_step = 0
        self.terminated = False
        self.reward = 0

        # Losujemy pozycję robota
        self.robot_x = np.random.randint(self.n)
        self.robot_y = np.random.randint(self.n)

        # Losujemy pozycję celu (inną niż robota)
        while True:
            self.target_x = np.random.randint(self.n)
            self.target_y = np.random.randint(self.n)
            if (self.target_x != self.robot_x) or (self.target_y != self.robot_y):
                break

        # Losujemy orientację
        self.orientation = np.random.randint(self.num_orientations)

        return self._get_obs()

    def step(self, action):
        """
        Zwraca krotkę (observation, reward, terminated, truncated, info).
        W Gym <=0.25 może to być (observation, reward, done, info).
        """
        self.reward = 0
        old_distance = np.linalg.norm([self.robot_x - self.target_x,
                                       self.robot_y - self.target_y])

        # Wykonanie akcji
        if action == 0:
            # Obrót w lewo o alpha
            self.orientation = (self.orientation - 1) % self.num_orientations
        elif action == 1:
            # Obrót w prawo o alpha
            self.orientation = (self.orientation + 1) % self.num_orientations
        elif action == 2:
            # Ruch do przodu w kierunku aktualnej orientacji
            self._move_forward()
        else:
            raise ValueError(f"Nieznana akcja: {action}")

        # Niewielka kara za każdy ruch (np. -0.0001)
        self.reward += -0.0001

        new_distance = np.linalg.norm([self.robot_x - self.target_x,
                                       self.robot_y - self.target_y])

        # Drobna nagroda za zbliżenie się do celu:
        if new_distance < old_distance:
            self.reward += 0.1
        elif new_distance > old_distance:
            # Kara za oddalenie się od celu
            self.reward -= 0.1

        # Sprawdzamy, czy dotarliśmy do celu
        if (self.robot_x == self.target_x) and (self.robot_y == self.target_y):
            self.reward = 1.0
            self.terminated = True

        # Inkrementacja kroku
        self.current_step += 1
        truncated = (self.current_step >= self.max_steps)

        return self._get_obs(), self.reward, self.terminated, truncated, {}

    def render(self, mode='human'):
        """
        Prosta wizualizacja w konsoli.
        'R' = robot, 'T' = cel, '.' = puste pole
        """
        grid = [["." for _ in range(self.n)] for _ in range(self.n)]

        # Robot
        grid[self.robot_y][self.robot_x] = "R"
        # Cel
        grid[self.target_y][self.target_x] = "T"

        print("=" * (2*self.n))
        for row in grid:
            print(" ".join(row))
        print("=" * (2*self.n))
        orientation_deg = self.orientation * self.alpha
        print(f"Orientacja: index={self.orientation}, kąt={orientation_deg}°")

    def _get_obs(self):
        """
        Zwraca obserwację:
           [robot_x, robot_y, target_x, target_y, orientation_index]
        """
        return np.array([
            self.robot_x,
            self.robot_y,
            self.target_x,
            self.target_y,
            self.orientation
        ], dtype=np.float32)

    def _move_forward(self):
        """
        Ruch robota do przodu o 1 kratkę w zależności od orientation.
        Jeśli ruch poza planszę -> kara i zakończenie epizodu.
        """
        angle_deg = self.orientation * self.alpha
        angle_rad = np.deg2rad(angle_deg)

        dx = int(round(np.cos(angle_rad)))
        dy = int(round(np.sin(angle_rad)))

        new_x = self.robot_x + dx
        new_y = self.robot_y + dy

        # Sprawdzenie granic
        if 0 <= new_x < self.n and 0 <= new_y < self.n:
            self.robot_x = new_x
            self.robot_y = new_y
        else:
            # Kara za wyjście poza obszar i zakończenie
            self.reward -= 0.5
            self.terminated = True

In [3]:
import gym
from gym import spaces
import numpy as np

class WarehouseGridEnv2(gym.Env):
    """
    Magazyn NxN z klasycznym sterowaniem (4 akcje):
      - Stan: [robot_x, robot_y, target_x, target_y]
      - Akcje: 0=góra, 1=dół, 2=lewo, 3=prawo

    Dodatkowo:
      - `grid` opisuje siatkę (n x n), w której 0 to wolne pole, 1 – pole zajęte.
      - Wejście na pole o wartości 1 traktujemy jak kolizję.
    """
    def __init__(self, 
                 n=5, 
                 max_steps=100, 
                 grid=None):
        super().__init__()

        self.max_steps = max_steps
        
        # Jeżeli przekazano siatkę, to z niej wyznacz rozmiar, w przeciwnym razie stwórz wolny obszar NxN
        if grid is not None:
            self.grid = np.array(grid, dtype=int)
            self.n = self.grid.shape[0]
        else:
            self.n = n
            # Domyślnie tworzymy wolną siatkę, same zera
            self.grid = np.zeros((self.n, self.n), dtype=int)

        # Przestrzeń akcji: 4
        self.action_space = spaces.Discrete(4)

        # Stan: (robot_x, robot_y, target_x, target_y)
        low = np.array([0, 0, 0, 0], dtype=np.float32)
        high = np.array([self.n - 1, self.n - 1, self.n - 1, self.n - 1], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        # Zmienna do śledzenia stanu (inicjalizowana w reset)
        self.robot_x = 0
        self.robot_y = 0
        self.target_x = 0
        self.target_y = 0
        self.current_step = 0
        self.terminated = False
        self.reward = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.terminated = False
        self.reward = 0

        # Losowanie pozycji robota (tylko na wolnym polu)
        while True:
            rx = np.random.randint(self.n)
            ry = np.random.randint(self.n)
            if self.grid[ry, rx] == 0:  # wolne pole
                self.robot_x = rx
                self.robot_y = ry
                break

        # Losowanie pozycji celu (tylko na wolnym polu i różne od pozycji robota)
        while True:
            tx = np.random.randint(self.n)
            ty = np.random.randint(self.n)
            if (self.grid[ty, tx] == 0 and
                (tx != self.robot_x or ty != self.robot_y)):
                self.target_x = tx
                self.target_y = ty
                break

        return self._get_obs()

    def step(self, action):
        old_distance = np.linalg.norm([self.robot_x - self.target_x,
                                       self.robot_y - self.target_y])

        # Określamy potencjalną nową pozycję
        if action == 0:   # góra
            new_x = self.robot_x
            new_y = self.robot_y - 1
        elif action == 1: # dół
            new_x = self.robot_x
            new_y = self.robot_y + 1
        elif action == 2: # lewo
            new_x = self.robot_x - 1
            new_y = self.robot_y
        elif action == 3: # prawo
            new_x = self.robot_x + 1
            new_y = self.robot_y
        else:
            raise ValueError(f"Nieznana akcja: {action}")

        # Sprawdzenie warunków kolizji lub wyjścia poza obszar:
        #  - wyjście poza obszar
        #  - pole zajęte (wartość 1 w grid)
        if not (0 <= new_x < self.n and 0 <= new_y < self.n):
            # Kara za wyjście poza obszar
            self.reward = -0.5
            self.terminated = True
        elif self.grid[new_y, new_x] == 1:
            # Kara za kolizję z polem zajętym
            self.reward = -0.5
            self.terminated = True
        else:
            # Jeżeli pole jest wolne, aktualizujemy pozycję
            self.robot_x = new_x
            self.robot_y = new_y
            # Domyślna, niewielka kara za ruch
            self.reward = -0.0001

            new_distance = np.linalg.norm([self.robot_x - self.target_x,
                                           self.robot_y - self.target_y])
            # Nagroda/kara za przybliżenie/oddalenie się od celu
            if new_distance < old_distance:
                self.reward += 0.5
            elif new_distance > old_distance:
                self.reward -= 0.5

        # Sprawdzamy, czy dotarliśmy do celu
        if (self.robot_x == self.target_x) and (self.robot_y == self.target_y):
            self.reward = 1.0
            self.terminated = True

        self.current_step += 1
        truncated = (self.current_step >= self.max_steps)

        return self._get_obs(), self.reward, self.terminated, truncated, {}

    def render(self, mode='human'):
        """
        Tekstowa wizualizacja środowiska:
        'R' = robot, 'T' = cel, 'X' = przeszkoda, '.' = puste pole.
        """
        grid_render = []
        for y in range(self.n):
            row_symbols = []
            for x in range(self.n):
                if self.grid[y, x] == 1:
                    symbol = "X"
                else:
                    symbol = "."

                if x == self.robot_x and y == self.robot_y:
                    symbol = "R"
                if x == self.target_x and y == self.target_y:
                    symbol = "T"
                row_symbols.append(symbol)
            grid_render.append(row_symbols)

        print("=" * (2 * self.n))
        for row in grid_render:
            print(" ".join(row))
        print("=" * (2 * self.n))

    def _get_obs(self):
        return np.array([
            self.robot_x, 
            self.robot_y,
            self.target_x, 
            self.target_y
        ], dtype=np.float32)


In [5]:
import gym
from gym import spaces
import numpy as np

class WarehouseGridEnv(gym.Env):
    """
    Magazyn NxN z klasycznym sterowaniem (4 akcje):
      - Stan: [robot_x, robot_y, target_x, target_y]
      - Akcje: 0=góra, 1=dół, 2=lewo, 3=prawo
    """
    def __init__(self, n=5, max_steps=100):
        super().__init__()  # wywołanie __init__ z gym.Env

        self.n = n
        self.max_steps = max_steps

        # Przestrzeń akcji: 4
        self.action_space = spaces.Discrete(4)

        # Stan: (robot_x, robot_y, target_x, target_y)
        low = np.array([0, 0, 0, 0], dtype=np.float32)
        high = np.array([n-1, n-1, n-1, n-1], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        self.robot_x = 0
        self.robot_y = 0
        self.target_x = 0
        self.target_y = 0
        self.current_step = 0
        self.terminated = False
        self.reward = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.terminated = False
        self.reward = 0

        # Losowanie pozycji robota
        self.robot_x = np.random.randint(self.n)
        self.robot_y = np.random.randint(self.n)

        # Losowanie pozycji celu
        while True:
            self.target_x = np.random.randint(self.n)
            self.target_y = np.random.randint(self.n)
            if (self.target_x != self.robot_x) or (self.target_y != self.robot_y):
                break

        return self._get_obs()

    def step(self, action):
        old_distance = np.linalg.norm([self.robot_x - self.target_x,
                                       self.robot_y - self.target_y])

        if action == 0:   # góra
            new_x = self.robot_x
            new_y = self.robot_y - 1
        elif action == 1: # dół
            new_x = self.robot_x
            new_y = self.robot_y + 1
        elif action == 2: # lewo
            new_x = self.robot_x - 1
            new_y = self.robot_y
        elif action == 3: # prawo
            new_x = self.robot_x + 1
            new_y = self.robot_y
        else:
            raise ValueError(f"Nieznana akcja: {action}")

        # Sprawdzenie granic
        if not (0 <= new_x < self.n and 0 <= new_y < self.n):
            # Kara za wyjście poza obszar
            self.reward = -0.5
            self.terminated = True
        else:
            # Aktualizacja pozycji
            self.robot_x = new_x
            self.robot_y = new_y
            # Domyślna niewielka kara za ruch
            self.reward = -0.0001

            new_distance = np.linalg.norm([self.robot_x - self.target_x,
                                           self.robot_y - self.target_y])
            # Nagroda/kara za przybliżenie/oddalenie
            if new_distance < old_distance:
                self.reward += 0.5
            elif new_distance > old_distance:
                self.reward -= 0.5

        # Sprawdzamy, czy dotarliśmy do celu
        if (self.robot_x == self.target_x) and (self.robot_y == self.target_y):
            self.reward = 1.0
            self.terminated = True

        self.current_step += 1
        truncated = (self.current_step >= self.max_steps)

        return self._get_obs(), self.reward, self.terminated, truncated, {}

    def render(self, mode='human'):
        """
        Tekstowa wizualizacja środowiska:
        'R' = robot, 'T' = cel, '.' = puste pole.
        """
        grid = [["." for _ in range(self.n)] for _ in range(self.n)]
        grid[self.robot_y][self.robot_x] = "R"
        grid[self.target_y][self.target_x] = "T"

        print("=" * (2 * self.n))
        for row in grid:
            print(" ".join(row))
        print("=" * (2 * self.n))

    def _get_obs(self):
        return np.array([self.robot_x, self.robot_y,
                         self.target_x, self.target_y], dtype=np.float32)


In [ ]:
import numpy as np
from tensorflow.keras.losses import MeanSquaredError
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from collections import deque
import random
import pickle
GAMMA = 0.9
EPSILON_START = 1.0
EPSILON_MIN = 0.0
EPSILON_DECAY = 0.99
BATCH_SIZE = 53164
LEARNING_RATE = 0.0001
MEMORY_SIZE = 20000000
EPISODES = 200000
def wczytaj_cyfry_z_pliku(nazwa_pliku):
    macierz = []
    with open(nazwa_pliku, 'r') as plik:
        for linia in plik:
            linia = linia.strip()
            # Zamiast split() możemy bezpośrednio iterować po znakach w linii
            # i konwertować je na int, o ile rzeczywiście mamy pojedyńcze cyfry
            wiersz = [int(znak) for znak in linia]
            macierz.append(wiersz)
    return macierz
my_grid=wczytaj_cyfry_z_pliku('map.txt')
# -----------------------------------------------------------------------
# Kluczowa rzecz: wybór środowiska, np. "orientation" lub "grid"
# -----------------------------------------------------------------------
ENV_TYPE = "orientation"  # "orientation" lub "grid"

if ENV_TYPE == "orientation":
    env=WarehouseOrientationEnv(n=64, alpha=90, max_steps=200)
else:
    my_grid =np.loadtxt('map.txt', dtype=int)
    env =WarehouseGridEnv2(n=4, max_steps=30)

# =======================================================================
# 3. Funkcje do transformacji stanu
# =======================================================================
def transform_state_orientation(obs):
    """
    Zakładamy obs ma postać (5,):
      [x, y, tx, ty, orientation]   gdzie orientation ∈ {0,1,2,3}
    Zwracamy [x, y, tx, ty, sin(θ), cos(θ)]
    """
    obs = np.array(obs, dtype=float)
    orientation = obs[4]
    angle_deg = orientation * 90.0
    angle_rad = np.deg2rad(angle_deg)
    sin_v = np.sin(angle_rad)
    cos_v = np.cos(angle_rad)
    new_obs = np.concatenate([obs[:4], [sin_v, cos_v]])
    return new_obs

def transform_state_grid(obs):
    """
    Zakładamy obs ma postać (4,):
      [x, y, tx, ty]
    Zwracamy identyczny wektor float (4,)
    """
    return np.array(obs, dtype=float)


# =======================================================================
# 4. Wybieramy odpowiednią transformację + ustalamy state_size
# =======================================================================
if ENV_TYPE == "orientation":
    transform_state_fn = transform_state_orientation
    state_size = 6  # bo mamy sin i cos dodatkowo
else:
    transform_state_fn = transform_state_grid
    state_size = 4  # (x, y, tx, ty)

action_size = env.action_space.n


# =======================================================================
# 5. Definicje modelu Q-network + target network
# =======================================================================
def build_q_network(state_size, action_size):
    model = models.Sequential()
    model.add(layers.Dense(64, input_shape=(state_size,), activation='softplus'))
    model.add(layers.Dense(64, input_shape=(state_size,), activation='softplus'))
    model.add(layers.Dense(64, input_shape=(state_size,), activation='softplus'))
    model.add(layers.Dense(action_size, activation='linear'))
    model.compile(optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
                  loss=MeanSquaredError())
    return model

def build_target_network(model):
    target = models.clone_model(model)
    target.set_weights(model.get_weights())
    return target


q_network = build_q_network(state_size, action_size)
target_network = build_target_network(q_network)

# =======================================================================
# 6. Wczytanie pamięci z pliku (opcjonalnie) lub stworzenie pustej
# =======================================================================
# try:
#     with open('memory.pkl', 'rb') as f:
#         memory = pickle.load(f)
#     print("Załadowano pamięć z pliku memory.pkl")
# except:
#     print("Nie znaleziono pliku memory.pkl, tworzę nową pamięć.")
#     memory = deque(maxlen=MEMORY_SIZE)
memory = deque(maxlen=MEMORY_SIZE)
epsilon = EPSILON_START
all_rewards = []
loss_value = 0.0

# =======================================================================
# 7. Główna pętla treningu
# =======================================================================
for episode in range(EPISODES):
    obs = env.reset()                      # Oryginalny stan z env (4D lub 5D)
    obs = transform_state_fn(obs)          # Transformacja do (4,) lub (6,)
    terminated = False
    truncated = False
    total_reward = 0

    while not (terminated or truncated):
        # Epsilon-greedy
        if np.random.rand() < epsilon or len(memory) < BATCH_SIZE:
            action = np.random.randint(action_size)
        else:
            q_values = q_network.predict(obs[np.newaxis, :], verbose=0)
            action = np.argmax(q_values[0])

        # Wykonanie akcji w środowisku
        next_obs, reward, terminated, truncated, info = env.step(action)
        next_obs = transform_state_fn(next_obs)

        total_reward += reward

        # Zapis do pamięci
        memory.append((obs, action, reward, next_obs, terminated))

        # Przechodzimy do nowej obserwacji
        obs = next_obs

    # Uczenie, jeśli mamy dostatecznie dużą pamięć
    if len(memory) >= BATCH_SIZE:
        minibatch = random.sample(memory, BATCH_SIZE)
        states_mb      = np.array([m[0] for m in minibatch])
        actions_mb     = np.array([m[1] for m in minibatch])
        rewards_mb     = np.array([m[2] for m in minibatch])
        next_states_mb = np.array([m[3] for m in minibatch])
        dones_mb       = np.array([m[4] for m in minibatch])

        q_next = q_network.predict(next_states_mb, verbose=0)
        q_next_target = target_network.predict(next_states_mb, verbose=0)
        q_targets = q_network.predict(states_mb, verbose=0)

        for i in range(BATCH_SIZE):
            if dones_mb[i]:
                q_targets[i, actions_mb[i]] = rewards_mb[i]
            else:
                a_max = np.argmax(q_next[i])
                q_targets[i, actions_mb[i]] = rewards_mb[i] + GAMMA * q_next_target[i, a_max]

        history = q_network.fit(states_mb, q_targets, epochs=1, verbose=0)
        loss_value = history.history['loss'][0]

        # (Można wprowadzić soft-update lub twarde kopiowanie co X epizodów)

    # Zmniejszamy epsilon
    if epsilon > EPSILON_MIN:
        epsilon *= EPSILON_DECAY

    all_rewards.append(total_reward)

    # Okresowe kopiowanie wag do sieci target
    if episode % 5 == 0:
        target_network.set_weights(q_network.get_weights())

    # Monitoring
    if (episode + 1) % 1 == 0:
        avg_reward = np.mean(all_rewards[-10:])
        print(f"Epizod: {episode+1}/{EPISODES}, "
              f"AVG ostatnich 10 ep.: {avg_reward:.3f}, "
              f"epsilon: {epsilon:.3f}, "
              f"loss: {loss_value:.5f}")

print("Trening zakończony!")

C:\Users\Filip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epizod: 1/200000, AVG ostatnich 10 ep.: 0.080, epsilon: 0.990, loss: 0.00000
Epizod: 2/200000, AVG ostatnich 10 ep.: -0.866, epsilon: 0.980, loss: 0.00000
Epizod: 3/200000, AVG ostatnich 10 ep.: -0.744, epsilon: 0.970, loss: 0.00000
Epizod: 4/200000, AVG ostatnich 10 ep.: -0.788, epsilon: 0.961, loss: 0.00000
Epizod: 5/200000, AVG ostatnich 10 ep.: -0.675, epsilon: 0.951, loss: 0.00000
Epizod: 6/200000, AVG ostatnich 10 ep.: -0.482, epsilon: 0.941, loss: 0.00000
Epizod: 7/200000, AVG ostatnich 10 ep.: -0.145, epsilon: 0.932, loss: 0.00000
Epizod: 8/200000, AVG ostatnich 10 ep.: 0.071, epsilon: 0.923, loss: 0.00000
Epizod: 9/200000, AVG ostatnich 10 ep.: -0.015, epsilon: 0.914, loss: 0.00000
Epizod: 10/200000, AVG ostatnich 10 ep.: 0.014, epsilon: 0.904, loss: 0.00000
Epizod: 11/200000, AVG ostatnich 10 ep.: 0.124, epsilon: 0.895, loss: 0.00000
Epizod: 12/200000, AVG ostatnich 10 ep.: 0.284, epsilon: 0.886, loss: 0.00000
Epizod: 13/200000, AVG ostatnich 10 ep.: 0.172, epsilon: 0.878, lo